# Import & Install packages

In [ ]:
import sys
import os

sys.path.append('../input/timm-pytorch-image-models/pytorch-image-models-master')
import timm
from timm.data import resolve_data_config
from timm.data.transforms_factory import create_transform

import pandas as pd
import numpy as np
import glob
import cv2
import matplotlib.pyplot as plt
import joblib
import gc
from glob import glob

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset

import timm
from PIL import Image
import PIL

from tqdm import tqdm
import joblib
import time
from tqdm.notebook import tqdm
import joblib
from sklearn.model_selection import StratifiedKFold

import cuml

import warnings

warnings.filterwarnings("ignore")

print(np.__version__)
print(pd.__version__)
print(torch.__version__)
print(timm.__version__)

In [2]:
#!pip install ../input/openaiclipweights/python-ftfy-master/python-ftfy-master
#!pip install ../input/openaiclipweights/clip/CLIP

In [3]:
!python --version

Python 3.7.12


# Load dataset

In [4]:
train = pd.read_csv('../input/petfinderdata/train-folds-1.csv')
test = pd.read_csv('../input/petfinder-pawpularity-score/test.csv')
sub = pd.read_csv('../input/petfinder-pawpularity-score/sample_submission.csv')

train['path'] = train['Id'].map(lambda x: '../input/petfinder-pawpularity-score/train/' + x + '.jpg')
test['path'] = test['Id'].map(lambda x: '../input/petfinder-pawpularity-score/test/' + x + '.jpg')

# If its Public LB run, then augment Testset to chack batch size memory consumption.
if test.shape[0] < 10:
    test = pd.concat([
        test, test, test, test, test,
    ])
    test = test.reset_index(drop=True)

print(train.shape, test.shape, sub.shape)

(9912, 18) (40, 14) (8, 2)


# Data Exploration

In [5]:
train['bins'] = (train['Pawpularity'] // 5).round()

train['fold0'] = -1
skf = StratifiedKFold(n_splits=20, shuffle=True, random_state=1)
for i, (_, test_index) in enumerate(skf.split(train.index, train['bins'])):
    train.iloc[test_index, -1] = i

train['fold0'] = train['fold0'].astype('int')
gc.collect()

train.groupby(['fold0'])['Pawpularity'].agg(['mean', 'std', 'count'])

,mean,std,count
fold0,,,
0,38.044355,20.623269,496
1,38.016129,20.599788,496
2,38.018145,20.700209,496
3,38.028226,20.665865,496
4,38.026210,20.756338,496
5,38.016129,20.668127,496
6,38.070565,20.590600,496
7,38.030242,20.731401,496
8,38.125000,20.565454,496


In [6]:
train.head()

,Id,Subject Focus,Eyes,Face,Near,Action,Accessory,Group,Collage,Human,Occlusion,Info,Blur,Pawpularity,fold,dup,nn0,path,bins,fold0
0,3ed899a8334a8e5c74f4a554f3ce6f08,0,1,1,1,0,0,0,0,0,0,0,0,1,1,-1,89a0b49aad4ce34c978b9a9c74cc95bd,../input/petfinder-pawpularity-score/train/3ed...,0,17
1,e0a1efdaf4fbed8659b6d23994ee346e,0,1,1,1,0,0,0,0,1,1,0,0,1,2,-2,b89199007b4fbd51864eeaf8f264f946,../input/petfinder-pawpularity-score/train/e0a...,0,14
2,6c159aede3df25fdbe781431aabcfc67,0,1,1,1,0,0,0,0,0,0,0,0,1,3,-3,59b3e4a3c4fe0f1f56cb7741100e7a30,../input/petfinder-pawpularity-score/train/6c1...,0,15
3,53b536999aecd800cfda720f3ca363cb,0,1,1,1,0,0,0,0,0,0,0,0,1,4,-4,0a05c55ca864b667d31c80ce2c68d6b3,../input/petfinder-pawpularity-score/train/53b...,0,19
4,4e3c8816d95b083b870c6747a26fcb58,0,0,0,1,0,0,0,0,1,1,0,1,2,5,-5,7af48bb7190c236d8876fe529b449c3f,../input/petfinder-pawpularity-score/train/4e3...,0,5


In [7]:
test.head()

,Id,Subject Focus,Eyes,Face,Near,Action,Accessory,Group,Collage,Human,Occlusion,Info,Blur,path
0,4128bae22183829d2b5fea10effdb0c3,1,0,1,0,0,1,1,0,0,1,0,1,../input/petfinder-pawpularity-score/test/4128...
1,43a2262d7738e3d420d453815151079e,0,1,0,0,0,0,1,1,0,0,0,0,../input/petfinder-pawpularity-score/test/43a2...
2,4e429cead1848a298432a0acad014c9d,0,0,0,1,0,1,1,1,0,1,1,1,../input/petfinder-pawpularity-score/test/4e42...
3,80bc3ccafcc51b66303c2c263aa38486,1,0,1,0,0,0,0,0,0,0,1,0,../input/petfinder-pawpularity-score/test/80bc...
4,8f49844c382931444e68dffbe20228f4,1,1,1,0,1,1,0,1,0,1,1,0,../input/petfinder-pawpularity-score/test/8f49...


# List Models

In [8]:
avail_pretrained_models = timm.list_models(pretrained=True)
len(avail_pretrained_models), avail_pretrained_models

(709,
 ['adv_inception_v3',
  'bat_resnext26ts',
  'beit_base_patch16_224',
  'beit_base_patch16_224_in22k',
  'beit_base_patch16_384',
  'beit_large_patch16_224',
  'beit_large_patch16_224_in22k',
  'beit_large_patch16_384',
  'beit_large_patch16_512',
  'botnet26t_256',
  'cait_m36_384',
  'cait_m48_448',
  'cait_s24_224',
  'cait_s24_384',
  'cait_s36_384',
  'cait_xs24_384',
  'cait_xxs24_224',
  'cait_xxs24_384',
  'cait_xxs36_224',
  'cait_xxs36_384',
  'coat_lite_mini',
  'coat_lite_small',
  'coat_lite_tiny',
  'coat_mini',
  'coat_tiny',
  'convit_base',
  'convit_small',
  'convit_tiny',
  'convmixer_768_32',
  'convmixer_1024_20_ks9_p14',
  'convmixer_1536_20',
  'convnext_base',
  'convnext_base_384_in22ft1k',
  'convnext_base_in22ft1k',
  'convnext_base_in22k',
  'convnext_large',
  'convnext_large_384_in22ft1k',
  'convnext_large_in22ft1k',
  'convnext_large_in22k',
  'convnext_small',
  'convnext_small_384_in22ft1k',
  'convnext_small_in22ft1k',
  'convnext_small_in22k',

#### Here we have listed all 575 pretrained model architectures available in timm library.

#### The first part of the solution is extracting the features from the last layer of listed models and run a SVR on the extracted features. 

#### Majority of the models are trained using 1000 classes in imagenet, which is why the output shae is 1000 for each model.

#### Idea is to find the subset od models (from 575) which has a good RMSE.

#### Forward models selection algorithm, followed by RMSE hill climbing logic was used. Starting with one model, and then keep adding until the RMSE stops incresing. 

#### The pretrained models found by the forward model selection alogirthm used in this solution is listed below. 

In [9]:
names = [
    'deit_base_distilled_patch16_384',
    'fbnetc_100',
    'ig_resnext101_32x8d',
    'ig_resnext101_32x48d',
    'repvgg_b0',
    'resnetv2_152x4_bitm',
    'rexnet_200',
    'resnest269e',
    'swsl_resnext101_32x8d',
    'tf_efficientnet_b6_ns',
    'tf_efficientnet_b7_ns',
    'tf_efficientnet_b8_ap',
    'tf_efficientnet_l2_ns_475',
    'vit_base_patch16_384',
    'vit_large_patch16_384',
    'vit_large_r50_s32_384',
]

names_hflip_crop = [
    'tf_efficientnet_l2_ns_hflip_384',
    'deit_base_distilled_patch16_384_hflip_384',
    'ig_resnext101_32x48d_hflip_384',
    'tf_efficientnet_l2_ns_512',
]

names_orig = [
    'ig_resnext101_32x48d',
    'vit_large_r50_s32_384'
]

### Dictionary with the path to all pretrained weights available in Kaggle datasets

In [10]:
modelpath = {m.split('/')[-1].split('.')[0]: m for m in
             glob('../input/pytorch-pretrained-0/*.pt') + glob('../input/pytorch-pretrained-1/*.pt') + glob(
                 '../input/pytorch-pretrained-2/*.pt') + glob('../input/pytorch-pretrained-3/*.pt')}
modelpath

{'resnetv2_101x1_bitm': '../input/pytorch-pretrained-0/resnetv2_101x1_bitm.pt',
 'xcit_large_24_p8_384_dist': '../input/pytorch-pretrained-0/xcit_large_24_p8_384_dist.pt',
 'tf_efficientnet_l2_ns_475': '../input/pytorch-pretrained-0/tf_efficientnet_l2_ns_475.pt',
 'swsl_resnext101_32x8d': '../input/pytorch-pretrained-0/swsl_resnext101_32x8d.pt',
 'ig_resnext101_32x48d': '../input/pytorch-pretrained-0/ig_resnext101_32x48d.pt',
 'ig_resnext101_32x8d': '../input/pytorch-pretrained-0/ig_resnext101_32x8d.pt',
 'vit_large_r50_s32_384': '../input/pytorch-pretrained-0/vit_large_r50_s32_384.pt',
 'vit_base_patch16_384': '../input/pytorch-pretrained-0/vit_base_patch16_384.pt',
 'deit_base_distilled_patch16_384': '../input/pytorch-pretrained-0/deit_base_distilled_patch16_384.pt',
 'beit_large_patch16_512': '../input/pytorch-pretrained-0/beit_large_patch16_512.pt',
 'rexnet_200': '../input/pytorch-pretrained-1/rexnet_200.pt',
 'resmlp_big_24_224_in22ft1k': '../input/pytorch-pretrained-1/resmlp_big

### Extracting the TESTSET features from each imagenet pretrained model and append to a dictionary.

In [11]:
class PawpularDataset:
    def __init__(self, images, base_path='../input/petfinder-pawpularity-score/train/', modelcfg=None, aug=0):
        self.images = images.copy()
        self.base_path = base_path
        self.transform = create_transform(**modelcfg)
        self.aug = aug

    def __len__(self):
        return len(self.images)

    def __getitem__(self, item):
        img = Image.open(self.base_path + self.images[item] + '.jpg').convert('RGB')
        img = self.transform(img)
        return img


EMB_TEST = {}
for arch in names:
    starttime = time.time()

    model = timm.create_model(arch, pretrained=False).to('cuda')
    model.load_state_dict(torch.load(modelpath[arch]))
    model.eval()

    train_dataset = PawpularDataset(
        images=test.Id.values,
        base_path='../input/petfinder-pawpularity-score/test/',
        modelcfg=resolve_data_config({}, model=model),
        aug=0,
    )
    BS = 10 if arch in ['tf_efficientnet_l2_ns'] else 16
    train_dataloader = DataLoader(train_dataset, batch_size=BS, num_workers=2, shuffle=False)

    with torch.no_grad():
        res = [model(img.to('cuda')).cpu().numpy() for img in train_dataloader]
    res = np.concatenate(res, 0)
    EMB_TEST[arch] = res

    print(arch, ', Done in:', int(time.time() - starttime), 's')

    del model, res
    torch.cuda.empty_cache()  # PyTorch thing to clean RAM
    gc.collect()

print(time.time())
len(EMB_TEST), EMB_TEST.keys()

deit_base_distilled_patch16_384 , Done in: 20 s
fbnetc_100 , Done in: 1 s
ig_resnext101_32x8d , Done in: 6 s
ig_resnext101_32x48d , Done in: 57 s
repvgg_b0 , Done in: 1 s
resnetv2_152x4_bitm , Done in: 69 s
rexnet_200 , Done in: 2 s
resnest269e , Done in: 12 s
swsl_resnext101_32x8d , Done in: 7 s
tf_efficientnet_b6_ns , Done in: 4 s
tf_efficientnet_b7_ns , Done in: 7 s
tf_efficientnet_b8_ap , Done in: 10 s
tf_efficientnet_l2_ns_475 , Done in: 40 s
vit_base_patch16_384 , Done in: 8 s
vit_large_patch16_384 , Done in: 27 s
vit_large_r50_s32_384 , Done in: 27 s
1670789962.867323


(16,
 dict_keys(['deit_base_distilled_patch16_384', 'fbnetc_100', 'ig_resnext101_32x8d', 'ig_resnext101_32x48d', 'repvgg_b0', 'resnetv2_152x4_bitm', 'rexnet_200', 'resnest269e', 'swsl_resnext101_32x8d', 'tf_efficientnet_b6_ns', 'tf_efficientnet_b7_ns', 'tf_efficientnet_b8_ap', 'tf_efficientnet_l2_ns_475', 'vit_base_patch16_384', 'vit_large_patch16_384', 'vit_large_r50_s32_384']))

In [12]:
EMB_TEST["deit_base_distilled_patch16_384"].shape

(40, 1000)

### We have 40 images in the test set, and 1000 classes.

### Extracting features using Horizontal Flip and small crop. 

In [13]:
class PawpularDataset_HFLIP:
    def __init__(self, images, base_path='../input/petfinder-pawpularity-score/train/', modelcfg=None, doflip=False):
        self.images = images.copy()
        self.base_path = base_path
        self.transform = modelcfg
        self.doflip = doflip

    def __len__(self):
        return len(self.images)

    def __getitem__(self, item):
        img = Image.open(self.base_path + self.images[item] + '.jpg').convert('RGB')

        if self.doflip == True:
            img = img.transpose(PIL.Image.FLIP_LEFT_RIGHT)
            width, height = img.size
            img = img.crop((0.0 * width, 0.02 * height, 0.98 * width, 0.98 * height))

        img = self.transform(img)
        return img


for arch in names_hflip_crop:
    starttime = time.time()

    archname = arch.split('_hflip_')[0]
    if arch == 'tf_efficientnet_l2_ns_512':
        archname = 'tf_efficientnet_l2_ns'
    model = timm.create_model(archname, pretrained=False).to('cuda')
    model.load_state_dict(torch.load(modelpath[archname]))
    model.eval()

    # Get model default transforms
    transf = resolve_data_config({}, model=model)
    sz = int(arch.split('_')[-1])
    transf['input_size'] = (3, sz, sz)
    transf['crop_pct'] = 1.0
    transf = create_transform(**transf)

    doflip = True if arch.split('_')[-2] == 'hflip' else False
    train_dataset = PawpularDataset_HFLIP(
        images=test.Id.values,
        base_path='../input/petfinder-pawpularity-score/test/',
        modelcfg=transf,
        doflip=doflip,
    )

    BS = 10 if archname in ['tf_efficientnet_l2_ns'] else 16
    train_dataloader = DataLoader(train_dataset, batch_size=BS, num_workers=2, shuffle=False)

    with torch.no_grad():
        res = [model(img.to('cuda')).cpu().numpy() for img in train_dataloader]
    res = np.concatenate(res, 0)
    EMB_TEST[arch] = res

    print(arch, 'imge size:', sz, 'Hflip:', doflip, ',Done in:', int(time.time() - starttime), 's')

    del model, res
    torch.cuda.empty_cache()  # PyTorch thing to clean RAM
    gc.collect()

print(time.time())
len(EMB_TEST), EMB_TEST.keys()

tf_efficientnet_l2_ns_hflip_384 imge size: 384 Hflip: True ,Done in: 38 s
deit_base_distilled_patch16_384_hflip_384 imge size: 384 Hflip: True ,Done in: 7 s
ig_resnext101_32x48d_hflip_384 imge size: 384 Hflip: True ,Done in: 56 s
tf_efficientnet_l2_ns_512 imge size: 512 Hflip: False ,Done in: 28 s
1670790094.502298


(20,
 dict_keys(['deit_base_distilled_patch16_384', 'fbnetc_100', 'ig_resnext101_32x8d', 'ig_resnext101_32x48d', 'repvgg_b0', 'resnetv2_152x4_bitm', 'rexnet_200', 'resnest269e', 'swsl_resnext101_32x8d', 'tf_efficientnet_b6_ns', 'tf_efficientnet_b7_ns', 'tf_efficientnet_b8_ap', 'tf_efficientnet_l2_ns_475', 'vit_base_patch16_384', 'vit_large_patch16_384', 'vit_large_r50_s32_384', 'tf_efficientnet_l2_ns_hflip_384', 'deit_base_distilled_patch16_384_hflip_384', 'ig_resnext101_32x48d_hflip_384', 'tf_efficientnet_l2_ns_512']))

### Loading TRAINSET extracted features (must be offline for Kaggle competitions)

In [14]:
EMB_TRAIN = joblib.load('../input/petfinderdata/train-embeddings-direct-1.joblib')
gc.collect()

resclip = joblib.load('../input/openai-clip/train-embeddings-openai-clip-1.joblib')
for m in resclip.keys():
    EMB_TRAIN[m] = resclip[m]
del resclip
gc.collect()

hflipmodels = joblib.load('../input/petfinder-extracted-pretrained-1/extracted-pretrained-1.joblib')
for col in names_hflip_crop:
    EMB_TRAIN[col] = hflipmodels[col]
del hflipmodels
gc.collect()

print(len(EMB_TRAIN))
print(EMB_TRAIN.keys())

58
dict_keys(['adv_inception_v3', 'cait_xs24_384', 'cspdarknet53', 'deit_base_distilled_patch16_384', 'deit_base_patch16_224', 'dla34', 'dm_nfnet_f2', 'dpn68', 'ese_vovnet19b_dw', 'fbnetc_100', 'hrnet_w18', 'hrnet_w30', 'ig_resnext101_32x8d', 'ig_resnext101_32x48d', 'mixnet_m', 'nfnet_l0', 'pit_ti_distilled_224', 'repvgg_b0', 'resnest101e', 'resnest200e', 'resnest269e', 'resnet34d', 'resnetv2_101x1_bitm', 'resnetv2_152x4_bitm', 'resnetv2_50x1_bitm', 'rexnet_130', 'rexnet_200', 'seresnet50', 'seresnet152d', 'ssl_resnext50_32x4d', 'swin_base_patch4_window7_224_in22k', 'swsl_resnext101_32x4d', 'swsl_resnext101_32x8d', 'tf_efficientnet_b5_ap', 'tf_efficientnet_b8_ap', 'tf_efficientnet_b6_ns', 'tf_efficientnet_b7_ns', 'tf_efficientnet_l2_ns_475', 'tf_efficientnetv2_l_in21ft1k', 'tf_efficientnet_l2_ns', 'tf_mixnet_s', 'twins_pcpvt_small', 'vit_base_patch16_384', 'vit_base_resnet50_384', 'vit_large_patch16_224', 'vit_large_patch16_384', 'vit_large_r50_s32_384', 'xception71', 'clip_RN50', 'cli

### Display Feature shapes

In [15]:
for m in EMB_TEST.keys():
    print(EMB_TRAIN[m].shape, EMB_TEST[m].shape, m)

(9912, 1000) (40, 1000) deit_base_distilled_patch16_384
(9912, 1000) (40, 1000) fbnetc_100
(9912, 1000) (40, 1000) ig_resnext101_32x8d
(9912, 1000) (40, 1000) ig_resnext101_32x48d
(9912, 1000) (40, 1000) repvgg_b0
(9912, 1000) (40, 1000) resnetv2_152x4_bitm
(9912, 1000) (40, 1000) rexnet_200
(9912, 1000) (40, 1000) resnest269e
(9912, 1000) (40, 1000) swsl_resnext101_32x8d
(9912, 1000) (40, 1000) tf_efficientnet_b6_ns
(9912, 1000) (40, 1000) tf_efficientnet_b7_ns
(9912, 1000) (40, 1000) tf_efficientnet_b8_ap
(9912, 1000) (40, 1000) tf_efficientnet_l2_ns_475
(9912, 1000) (40, 1000) vit_base_patch16_384
(9912, 1000) (40, 1000) vit_large_patch16_384
(9912, 1000) (40, 1000) vit_large_r50_s32_384
(9912, 1000) (40, 1000) tf_efficientnet_l2_ns_hflip_384
(9912, 1000) (40, 1000) deit_base_distilled_patch16_384_hflip_384
(9912, 1000) (40, 1000) ig_resnext101_32x48d_hflip_384
(9912, 1000) (40, 1000) tf_efficientnet_l2_ns_512


In [16]:
names0 = [

    'deit_base_distilled_patch16_384',
    'ig_resnext101_32x48d',
    'repvgg_b0',
    'resnetv2_152x4_bitm',
    'swsl_resnext101_32x8d',
    'tf_efficientnet_l2_ns_475',
    'vit_base_patch16_384',
    'vit_large_r50_s32_384'
]

names1 = [

    'fbnetc_100',
    'ig_resnext101_32x8d',
    'rexnet_200',
    'resnest269e',
    'tf_efficientnet_b6_ns',
    'tf_efficientnet_b8_ap',
    'tf_efficientnet_b7_ns',
    'vit_large_patch16_384'
]

names2 = [
    'tf_efficientnet_l2_ns_hflip_384',
    'deit_base_distilled_patch16_384_hflip_384',
    'ig_resnext101_32x48d_hflip_384',
    'tf_efficientnet_l2_ns_512',
    'ig_resnext101_32x48d',
    'vit_large_r50_s32_384'
]

names = np.unique(names0 + names1 + names2)
len(names), names

(20,
 array(['deit_base_distilled_patch16_384',
        'deit_base_distilled_patch16_384_hflip_384', 'fbnetc_100',
        'ig_resnext101_32x48d', 'ig_resnext101_32x48d_hflip_384',
        'ig_resnext101_32x8d', 'repvgg_b0', 'resnest269e',
        'resnetv2_152x4_bitm', 'rexnet_200', 'swsl_resnext101_32x8d',
        'tf_efficientnet_b6_ns', 'tf_efficientnet_b7_ns',
        'tf_efficientnet_b8_ap', 'tf_efficientnet_l2_ns_475',
        'tf_efficientnet_l2_ns_512', 'tf_efficientnet_l2_ns_hflip_384',
        'vit_base_patch16_384', 'vit_large_patch16_384',
        'vit_large_r50_s32_384'], dtype='<U41'))

### Clean memory of offline TRAINSET features which are not going to be used

In [17]:
feats = list(EMB_TRAIN.keys())
for n in feats:
    if n not in names:
        del EMB_TRAIN[n]
        gc.collect()

### GPU accelerated SVR using cuML

In [18]:
from cuml.svm import SVR
from sklearn.preprocessing import StandardScaler


def fit_gpu_svr(TRAIN, TEST, kfoldcol='fold0'):
    ypredtrain_ = np.zeros(train.shape[0])
    ypredtest_ = np.zeros(test.shape[0])

    for fold in range(train[kfoldcol].max() + 1):
        ind_train = train[kfoldcol] != fold
        ind_valid = train[kfoldcol] == fold

        model = SVR(C=30.0, kernel='rbf', degree=3, max_iter=4000, output_type='numpy')
        model.fit(TRAIN[ind_train], train.Pawpularity[ind_train].clip(1, 85))

        ypredtrain_[ind_valid] = np.clip(model.predict(TRAIN[ind_valid]), 1, 100)
        ypredtest_ += np.clip(model.predict(TEST), 1, 100)

        del model
        gc.collect()

    ypredtest_ /= (train[kfoldcol].max() + 1)

    return ypredtrain_, ypredtest_


def rmse(ytrue, ypred):
    return np.sqrt(np.mean((ytrue - ypred) ** 2))

### SVR for each architecture

In [19]:
for col in names:
    TRAIN = EMB_TRAIN[col].copy()
    TEST = EMB_TEST[col].copy()

    scaler = StandardScaler()
    scaler.fit(np.vstack((TRAIN, TEST)))
    TRAIN = scaler.transform(TRAIN)
    TEST = scaler.transform(TEST)

    ypredtrain, ypredtest = fit_gpu_svr(TRAIN, TEST, 'fold0')
    print(rmse(train.Pawpularity, ypredtrain), col)

17.712809941265807 deit_base_distilled_patch16_384
17.704078387405332 deit_base_distilled_patch16_384_hflip_384
18.48424837093278 fbnetc_100
17.755805285329178 ig_resnext101_32x48d
17.780893496477624 ig_resnext101_32x48d_hflip_384
17.958093335328353 ig_resnext101_32x8d
18.253444602567463 repvgg_b0
18.0009617924418 resnest269e
18.13575249613704 resnetv2_152x4_bitm
17.973611293382902 rexnet_200
18.015502770431883 swsl_resnext101_32x8d
17.744341478237462 tf_efficientnet_b6_ns
17.742343658651897 tf_efficientnet_b7_ns
17.750993426678342 tf_efficientnet_b8_ap
17.619187153498366 tf_efficientnet_l2_ns_475
17.61844972450698 tf_efficientnet_l2_ns_512
17.689217431423767 tf_efficientnet_l2_ns_hflip_384
17.92158811171319 vit_base_patch16_384
18.01499760636718 vit_large_patch16_384
18.00659110030843 vit_large_r50_s32_384


### As we can see above, SVR RMSE ranges from 17.56 to 18.52

### Idea is to stack architectures features together

### SVR A (names0)

In [20]:
print('Concatenating:', names0)

TRAIN = np.concatenate([EMB_TRAIN[k] for k in names0], 1)
TEST = np.concatenate([EMB_TEST[k] for k in names0], 1)
scaler = StandardScaler()
scaler.fit(np.vstack((TRAIN, TEST)))
gc.collect()

TRAIN = scaler.transform(TRAIN)
TEST = scaler.transform(TEST)
gc.collect()

# Check the output shape
print(TRAIN.shape, TEST.shape)

ypredtrainA, ypredtestA = fit_gpu_svr(TRAIN, TEST, 'fold0')
print(rmse(train.Pawpularity, ypredtrainA))

Concatenating: ['deit_base_distilled_patch16_384', 'ig_resnext101_32x48d', 'repvgg_b0', 'resnetv2_152x4_bitm', 'swsl_resnext101_32x8d', 'tf_efficientnet_l2_ns_475', 'vit_base_patch16_384', 'vit_large_r50_s32_384']
(9912, 8000) (40, 8000)
17.231539198315136


In [21]:
del TRAIN, TEST
gc.collect()
torch.cuda.empty_cache()  # PyTorch thing

### Using the multiplier of 1.032 boosts RMSE score, probably because SVR uses MSE based optimization. 

In [22]:
print('RMSE:', rmse(train.Pawpularity, 1.032 * ypredtrainA))

RMSE: 17.224564776006655


### SVR B (names1)

In [23]:
print('Concatenating:', names1)

TRAIN = np.concatenate([EMB_TRAIN[k] for k in names1], 1)
TEST = np.concatenate([EMB_TEST[k] for k in names1], 1)
scaler = StandardScaler()
scaler.fit(np.vstack((TRAIN, TEST)))
gc.collect()

TRAIN = scaler.transform(TRAIN)
TEST = scaler.transform(TEST)
gc.collect()

print(TRAIN.shape, TEST.shape)

ypredtrainB, ypredtestB = fit_gpu_svr(TRAIN, TEST, 'fold0')
print('RMSE:', rmse(train.Pawpularity, ypredtrainB))

Concatenating: ['fbnetc_100', 'ig_resnext101_32x8d', 'rexnet_200', 'resnest269e', 'tf_efficientnet_b6_ns', 'tf_efficientnet_b8_ap', 'tf_efficientnet_b7_ns', 'vit_large_patch16_384']
(9912, 8000) (40, 8000)
RMSE: 17.337306863761512


In [24]:
del TRAIN, TEST
gc.collect()
torch.cuda.empty_cache()  # PyTorch thing

### SVR C (names2)

In [25]:
print('Concatenating:', names2)

TRAIN = np.concatenate([EMB_TRAIN[k] for k in names2], 1)
TEST = np.concatenate([EMB_TEST[k] for k in names2], 1)
scaler = StandardScaler()
scaler.fit(np.vstack((TRAIN, TEST)))
gc.collect()

TRAIN = scaler.transform(TRAIN)
TEST = scaler.transform(TEST)
gc.collect()

print(TRAIN.shape, TEST.shape)

ypredtrainC, ypredtestC = fit_gpu_svr(TRAIN, TEST, 'fold0')
print('RMSE:', rmse(train.Pawpularity, ypredtrainC))

Concatenating: ['tf_efficientnet_l2_ns_hflip_384', 'deit_base_distilled_patch16_384_hflip_384', 'ig_resnext101_32x48d_hflip_384', 'tf_efficientnet_l2_ns_512', 'ig_resnext101_32x48d', 'vit_large_r50_s32_384']
(9912, 6000) (40, 6000)
RMSE: 17.263647505975836


In [26]:
# Free RAM and GPU memory
del TRAIN, TEST
del EMB_TRAIN, EMB_TEST
gc.collect()

torch.cuda.empty_cache()  # PyTorch thing to free GPU memory
gc.collect()

0

### Run inference using Deep Learning finetuned image models. 

In [27]:
from torch.utils.data import Dataset, DataLoader
import albumentations as A

device = torch.device('cuda')


class Config:
    model_name = "swin_large_patch4_window7_224"
    base_dir = "../input/petfinder-pawpularity-score"
    data_dir = base_dir
    model_dir = "exp"
    output_dir = model_dir
    img_test_dir = os.path.join(data_dir, "test")
    model_path = "swin_large_patch4_window7_224"
    im_size = 384
    batch_size = 16


class PetDataset(Dataset):
    def __init__(self, image_filepaths, targets, transform=None):
        self.image_filepaths = image_filepaths
        self.targets = targets
        self.transform = transform

    def __len__(self):
        return len(self.image_filepaths)

    def __getitem__(self, idx):
        image_filepath = self.image_filepaths[idx]
        with open(image_filepath, 'rb') as f:
            image = Image.open(f)
            image_rgb = image.convert('RGB')
        image = np.array(image_rgb)

        if self.transform is not None:
            image = self.transform(image=image)["image"]

        image = image / 255
        image = np.transpose(image, (2, 0, 1)).astype(np.float32)
        target = self.targets[idx]

        image = torch.tensor(image, dtype=torch.float)
        target = torch.tensor(target, dtype=torch.float)
        return image, target


def get_inference_fixed_transforms(mode=0, dim=224):
    if mode == 0:  # do not original aspects, colors and angles
        return A.Compose([
            A.SmallestMaxSize(max_size=dim, p=1.0),
            A.CenterCrop(height=dim, width=dim, p=1.0),
        ], p=1.0)
    elif mode == 1:
        return A.Compose([
            A.SmallestMaxSize(max_size=dim + 16, p=1.0),
            A.CenterCrop(height=dim, width=dim, p=1.0),
            A.HorizontalFlip(p=1.0)
        ], p=1.0)


class PetNet(nn.Module):
    def __init__(
            self,
            model_name=Config.model_path,
            out_features=1,
            inp_channels=3,
            pretrained=False,
    ):
        super().__init__()
        self.model = timm.create_model(model_name, pretrained=False, in_chans=3, num_classes=1)

    def forward(self, image):
        output = self.model(image)
        return output


def tta_fn(filepaths, model, ttas=[0, 1]):
    print('Image Size:', Config.im_size)
    model.eval()
    tta_preds = []
    for tta_mode in ttas:  #range(Config.tta_times):
        print(f'tta mode:{tta_mode}')
        test_dataset = PetDataset(
            image_filepaths=filepaths,
            targets=np.zeros(len(filepaths)),
            transform=get_inference_fixed_transforms(tta_mode, dim=Config.im_size)
        )
        test_loader = DataLoader(
            test_dataset,
            batch_size=Config.batch_size,
            shuffle=False,
            num_workers=2,
            pin_memory=True
        )
        #stream = tqdm(test_loader)
        tta_pred = []
        for images, target in test_loader:  #enumerate(stream, start = 1):
            images = images.to(device, non_blocking=True).float()
            target = target.to(device, non_blocking=True).float().view(-1, 1)
            with torch.no_grad():
                output = model(images)

            pred = (torch.sigmoid(output).detach().cpu().numpy() * 100).ravel().tolist()
            tta_pred.extend(pred)
        tta_preds.append(np.array(tta_pred))

    fold_preds = tta_preds[0]
    for n in range(1, len(tta_preds)):
        fold_preds += tta_preds[n]
    fold_preds /= len(tta_preds)

    del test_loader, test_dataset
    gc.collect()
    torch.cuda.empty_cache()
    return fold_preds

In [28]:
# List all test files
filepaths = test['path'].values.copy()
len(filepaths)

40

In [29]:
% % time

class Config:
    model_dir = "exp53"
    output_dir = "exp53"
    model_name = "swin_large_patch4_window7_224"
    im_size = 224
    model_path = model_name
    base_dir = "../input/petfinder-pawpularity-score"
    data_dir = base_dir
    img_test_dir = os.path.join(data_dir, "test")
    batch_size = 16


test_preds = []
test_preds_model = []
modelfiles = glob('../input/petfinder-' + Config.model_dir + '/*.pth')
for mi, model_path in enumerate(modelfiles):
    print(f'inference: {model_path}')
    test_preds_fold = []
    model = PetNet(
        model_name=Config.model_path,
        out_features=1,
        inp_channels=3,
        pretrained=False
    )
    model.load_state_dict(torch.load(model_path))
    model = model.to(device)
    model = model.float()
    model.eval()
    test_preds_fold = tta_fn(filepaths, model, [1])
    test_preds_model.append(test_preds_fold)

oof53 = pd.read_csv('../input/petfinder-' + Config.model_dir + '/oof_tta.csv')
final_predictions53 = np.mean(np.array(test_preds_model), axis=0)
final_predictions53

inference: ../input/petfinder-exp53/swin_large_patch4_window7_224_fold5_half.pth
Image Size: 224
tta mode:1
inference: ../input/petfinder-exp53/swin_large_patch4_window7_224_fold9_half.pth
Image Size: 224
tta mode:1
inference: ../input/petfinder-exp53/swin_large_patch4_window7_224_fold2_half.pth
Image Size: 224
tta mode:1
inference: ../input/petfinder-exp53/swin_large_patch4_window7_224_fold1_half.pth
Image Size: 224
tta mode:1
inference: ../input/petfinder-exp53/swin_large_patch4_window7_224_fold0_half.pth
Image Size: 224
tta mode:1
inference: ../input/petfinder-exp53/swin_large_patch4_window7_224_fold6_half.pth
Image Size: 224
tta mode:1
inference: ../input/petfinder-exp53/swin_large_patch4_window7_224_fold4_half.pth
Image Size: 224
tta mode:1
inference: ../input/petfinder-exp53/swin_large_patch4_window7_224_fold7_half.pth
Image Size: 224
tta mode:1
inference: ../input/petfinder-exp53/swin_large_patch4_window7_224_fold8_half.pth
Image Size: 224
tta mode:1
inference: ../input/petfinde

array([41.54221554, 42.02376919, 42.01871738, 42.70148811, 41.79610214,
       42.11279564, 42.43319626, 41.76821747, 41.54221554, 42.02376919,
       42.01871738, 42.70148811, 41.79610214, 42.11279564, 42.43319626,
       41.76821747, 41.54221554, 42.02376919, 42.01871738, 42.70148811,
       41.79610214, 42.11279564, 42.43319626, 41.76821747, 41.54221554,
       42.02376919, 42.01871738, 42.70148811, 41.79610214, 42.11279564,
       42.43319626, 41.76821747, 41.54221344, 42.02376728, 42.01871548,
       42.70148315, 41.79609795, 42.11279297, 42.43319435, 41.76821861])

In [30]:
% % time

class Config:
    model_dir = "exp55"
    output_dir = "exp55"
    model_name = "beit_large_patch16_224"
    im_size = 224
    model_path = model_name
    base_dir = "../input/petfinder-pawpularity-score"
    data_dir = base_dir
    img_test_dir = os.path.join(data_dir, "test")
    batch_size = 16


test_preds = []
test_preds_model = []
modelfiles = glob('../input/petfinder-' + Config.model_dir + '/*.pth')
for mi, model_path in enumerate(modelfiles):
    print(f'inference: {model_path}')
    test_preds_fold = []
    model = PetNet(
        model_name=Config.model_path,
        out_features=1,
        inp_channels=3,
        pretrained=False
    )
    model.load_state_dict(torch.load(model_path))
    model = model.to(device)
    model = model.float()
    model.eval()
    test_preds_fold = tta_fn(filepaths, model, [0])
    test_preds_model.append(test_preds_fold)

oof55 = pd.read_csv('../input/petfinder-' + Config.model_dir + '/oof_tta.csv')
final_predictions55 = np.mean(np.array(test_preds_model), axis=0)
final_predictions55

inference: ../input/petfinder-exp55/beit_large_patch16_224_fold4_half.pth
Image Size: 224
tta mode:0
inference: ../input/petfinder-exp55/beit_large_patch16_224_fold7_half.pth
Image Size: 224
tta mode:0
inference: ../input/petfinder-exp55/beit_large_patch16_224_fold1_half.pth
Image Size: 224
tta mode:0
inference: ../input/petfinder-exp55/beit_large_patch16_224_fold0_half.pth
Image Size: 224
tta mode:0
inference: ../input/petfinder-exp55/beit_large_patch16_224_fold5_half.pth
Image Size: 224
tta mode:0
inference: ../input/petfinder-exp55/beit_large_patch16_224_fold9_half.pth
Image Size: 224
tta mode:0
inference: ../input/petfinder-exp55/beit_large_patch16_224_fold6_half.pth
Image Size: 224
tta mode:0
inference: ../input/petfinder-exp55/beit_large_patch16_224_fold2_half.pth
Image Size: 224
tta mode:0
inference: ../input/petfinder-exp55/beit_large_patch16_224_fold8_half.pth
Image Size: 224
tta mode:0
inference: ../input/petfinder-exp55/beit_large_patch16_224_fold3_half.pth
Image Size: 224
t

array([48.3031147 , 48.46970367, 48.34081039, 48.52948532, 48.60842323,
       48.04617119, 48.40596275, 48.20590172, 48.3031147 , 48.46970367,
       48.34081039, 48.52948532, 48.60842323, 48.04617119, 48.40596275,
       48.20590172, 48.3031147 , 48.46970367, 48.34081039, 48.52948532,
       48.60842323, 48.04617119, 48.40596275, 48.20590172, 48.3031147 ,
       48.46970367, 48.34081039, 48.52948532, 48.60842323, 48.04617119,
       48.40596275, 48.20590172, 48.30311203, 48.46970329, 48.34081154,
       48.5294899 , 48.60842361, 48.04617386, 48.40596008, 48.20590057])

In [31]:
% % time

class Config:
    model_dir = "exp66"
    output_dir = "exp66"
    model_name = "swin_large_patch4_window12_384_in22k"
    im_size = 384
    model_path = model_name
    base_dir = "../input/petfinder-pawpularity-score"
    data_dir = base_dir
    img_test_dir = os.path.join(data_dir, "test")
    batch_size = 16


test_preds = []
test_preds_model = []
modelfiles = glob('../input/petfinder-' + Config.model_dir + '/*.pth')
for mi, model_path in enumerate(modelfiles):
    print(f'inference: {model_path}')
    test_preds_fold = []
    model = PetNet(
        model_name=Config.model_path,
        out_features=1,
        inp_channels=3,
        pretrained=False
    )
    model.load_state_dict(torch.load(model_path))
    model = model.to(device)
    model = model.float()
    model.eval()
    test_preds_fold = tta_fn(filepaths, model, [0])
    test_preds_model.append(test_preds_fold)

oof66 = pd.read_csv('../input/petfinder-' + Config.model_dir + '/oof_tta.csv')
final_predictions66 = np.mean(np.array(test_preds_model), axis=0)
final_predictions66

inference: ../input/petfinder-exp66/swin_large_patch4_window12_384_in22k_fold5_half.pth
Image Size: 384
tta mode:0
inference: ../input/petfinder-exp66/swin_large_patch4_window12_384_in22k_fold2_half.pth
Image Size: 384
tta mode:0
inference: ../input/petfinder-exp66/swin_large_patch4_window12_384_in22k_fold8_half.pth
Image Size: 384
tta mode:0
inference: ../input/petfinder-exp66/swin_large_patch4_window12_384_in22k_fold7_half.pth
Image Size: 384
tta mode:0
inference: ../input/petfinder-exp66/swin_large_patch4_window12_384_in22k_fold1_half.pth
Image Size: 384
tta mode:0
inference: ../input/petfinder-exp66/swin_large_patch4_window12_384_in22k_fold3_half.pth
Image Size: 384
tta mode:0
inference: ../input/petfinder-exp66/swin_large_patch4_window12_384_in22k_fold0_half.pth
Image Size: 384
tta mode:0
inference: ../input/petfinder-exp66/swin_large_patch4_window12_384_in22k_fold6_half.pth
Image Size: 384
tta mode:0
inference: ../input/petfinder-exp66/swin_large_patch4_window12_384_in22k_fold9_h

array([44.19167061, 43.99319916, 44.02210159, 44.39536629, 43.86056747,
       44.02656174, 44.71376114, 44.19404564, 44.19167061, 43.99319916,
       44.02210159, 44.39536629, 43.86056747, 44.02656174, 44.71376114,
       44.19404564, 44.19167061, 43.99319916, 44.02210159, 44.39536629,
       43.86056747, 44.02656174, 44.71376114, 44.19404564, 44.19167061,
       43.99319916, 44.02210159, 44.39536629, 43.86056747, 44.02656174,
       44.71376114, 44.19404564, 44.19167213, 43.99320107, 44.02210236,
       44.39536781, 43.86056786, 44.02656097, 44.71376114, 44.19404488])

In [32]:
% % time

class Config:
    model_dir = "exp77"
    output_dir = "exp77"
    model_name = "beit_large_patch16_224"
    im_size = 224
    model_path = model_name
    base_dir = "../input/petfinder-pawpularity-score"
    data_dir = base_dir
    img_test_dir = os.path.join(data_dir, "test")
    batch_size = 16


class PetNet(nn.Module):
    def __init__(
            self,
            model_name=Config.model_path,
            out_features=1,
            inp_channels=3,
            pretrained=False
    ):
        super().__init__()
        NC = 1000
        self.model = timm.create_model(model_name, pretrained=False)
        self.dropout = nn.Dropout(0.05)
        self.head = nn.Linear(NC, 1)

    def forward(self, image):
        output = self.model(image)
        output = self.dropout(output)
        output = self.head(output)
        return output


test_preds = []
test_preds_model = []
modelfiles = glob('../input/petfinder-' + Config.model_dir + '/*.pth')
for mi, model_path in enumerate(modelfiles):
    print(f'inference: {model_path}')
    test_preds_fold = []
    model = PetNet(
        model_name=Config.model_path,
        out_features=1,
        inp_channels=3,
        pretrained=False
    )
    model.load_state_dict(torch.load(model_path))
    model = model.to(device)
    model = model.float()
    model.eval()
    test_preds_fold = tta_fn(filepaths, model, [0])
    test_preds_model.append(test_preds_fold)

oof77 = pd.read_csv('../input/petfinder-' + Config.model_dir + '/oof_tta.csv')
final_predictions77 = np.mean(np.array(test_preds_model), axis=0)
final_predictions77

inference: ../input/petfinder-exp77/beit_large_patch16_224_fold4_half.pth
Image Size: 224
tta mode:0
inference: ../input/petfinder-exp77/beit_large_patch16_224_fold7_half.pth
Image Size: 224
tta mode:0
inference: ../input/petfinder-exp77/beit_large_patch16_224_fold1_half.pth
Image Size: 224
tta mode:0
inference: ../input/petfinder-exp77/beit_large_patch16_224_fold0_half.pth
Image Size: 224
tta mode:0
inference: ../input/petfinder-exp77/beit_large_patch16_224_fold5_half.pth
Image Size: 224
tta mode:0
inference: ../input/petfinder-exp77/beit_large_patch16_224_fold9_half.pth
Image Size: 224
tta mode:0
inference: ../input/petfinder-exp77/beit_large_patch16_224_fold6_half.pth
Image Size: 224
tta mode:0
inference: ../input/petfinder-exp77/beit_large_patch16_224_fold2_half.pth
Image Size: 224
tta mode:0
inference: ../input/petfinder-exp77/beit_large_patch16_224_fold8_half.pth
Image Size: 224
tta mode:0
inference: ../input/petfinder-exp77/beit_large_patch16_224_fold3_half.pth
Image Size: 224
t

array([44.79068413, 45.35668354, 45.35797005, 45.44356155, 46.04086342,
       45.89144001, 46.58387165, 45.84169559, 44.79068413, 45.35668354,
       45.35797005, 45.44356155, 46.04086342, 45.89144001, 46.58387165,
       45.84169559, 44.79068413, 45.35668354, 45.35797005, 45.44356155,
       46.04086342, 45.89144001, 46.58387165, 45.84169559, 44.79068413,
       45.35668354, 45.35797005, 45.44356155, 46.04086342, 45.89144001,
       46.58387165, 45.84169559, 44.79068184, 45.35668907, 45.35796356,
       45.44356709, 46.0408659 , 45.89143333, 46.58385677, 45.8416872 ])

In [33]:
% % time

class Config:
    model_dir = "exp82"
    output_dir = "exp82"
    model_name = "tf_efficientnet_b6_ns"
    im_size = 528
    model_path = model_name
    base_dir = "../input/petfinder-pawpularity-score"
    data_dir = base_dir
    img_test_dir = os.path.join(data_dir, "test")
    batch_size = 16


class PetNet(nn.Module):
    def __init__(
            self,
            model_name=Config.model_path,
            out_features=1,
            inp_channels=3,
            pretrained=False
    ):
        super().__init__()
        NC = 1000
        self.model = timm.create_model(model_name, pretrained=False)
        self.dropout = nn.Dropout(0.25)
        self.head = nn.Linear(NC, out_features)

    def forward(self, image):
        output = self.model(image)
        output = self.dropout(output)
        output = self.head(output)
        return output


test_preds = []
test_preds_model = []
modelfiles = glob('../input/petfinder-' + Config.model_dir + '/*.pth')
for mi, model_path in enumerate(modelfiles):
    print(f'inference: {model_path}')
    test_preds_fold = []
    model = PetNet(
        model_name=Config.model_path,
        out_features=1,
        inp_channels=3,
        pretrained=False
    )
    model.load_state_dict(torch.load(model_path))
    model = model.to(device)
    model = model.float()
    model.eval()
    test_preds_fold = tta_fn(filepaths, model, [0])
    test_preds_model.append(test_preds_fold)

oof82 = pd.read_csv('../input/petfinder-' + Config.model_dir + '/oof_tta.csv')
final_predictions82 = np.mean(np.array(test_preds_model), axis=0)
final_predictions82

inference: ../input/petfinder-exp82/tf_efficientnet_b6_ns_fold3_half.pth
Image Size: 528
tta mode:0
inference: ../input/petfinder-exp82/tf_efficientnet_b6_ns_fold7_half.pth
Image Size: 528
tta mode:0
inference: ../input/petfinder-exp82/tf_efficientnet_b6_ns_fold6_half.pth
Image Size: 528
tta mode:0
inference: ../input/petfinder-exp82/tf_efficientnet_b6_ns_fold5_half.pth
Image Size: 528
tta mode:0
inference: ../input/petfinder-exp82/tf_efficientnet_b6_ns_fold8_half.pth
Image Size: 528
tta mode:0
inference: ../input/petfinder-exp82/tf_efficientnet_b6_ns_fold0_half.pth
Image Size: 528
tta mode:0
inference: ../input/petfinder-exp82/tf_efficientnet_b6_ns_fold9_half.pth
Image Size: 528
tta mode:0
inference: ../input/petfinder-exp82/tf_efficientnet_b6_ns_fold1_half.pth
Image Size: 528
tta mode:0
inference: ../input/petfinder-exp82/tf_efficientnet_b6_ns_fold4_half.pth
Image Size: 528
tta mode:0
inference: ../input/petfinder-exp82/tf_efficientnet_b6_ns_fold2_half.pth
Image Size: 528
tta mode:0


array([37.48838024, 37.35873871, 36.76660423, 36.27986965, 37.43988914,
       37.87288513, 36.98650017, 37.08287888, 37.48838024, 37.35873871,
       36.76660423, 36.27986965, 37.43988914, 37.87288513, 36.98650017,
       37.08287888, 37.48838024, 37.35873871, 36.76660423, 36.27986965,
       37.43988914, 37.87288513, 36.98650017, 37.08287888, 37.48838024,
       37.35873871, 36.76660423, 36.27986965, 37.43988914, 37.87288513,
       36.98650017, 37.08287888, 37.48838024, 37.35873871, 36.76660423,
       36.27986965, 37.43988914, 37.87288513, 36.98650017, 37.08287888])

In [34]:
# Set the weights

a = 3
b = 4
c = 3
d = 4
e = 2

### Weighted average image models

In [35]:
oof = oof53.copy()
oof['pred'] = (
                      a * oof53['pred'] +
                      b * oof55['pred'] +
                      c * oof66['pred'] +
                      d * oof77['pred'] +
                      e * oof82['pred']
              ) / (a + b + c + d + e)

final_train_predictions = train.merge(oof, on='Id', how='left')['pred'].values.copy()

rmse(train.Pawpularity.values, final_train_predictions)

17.02954258971035

In [36]:
final_test_predictions = (
                                 a * final_predictions53 +
                                 b * final_predictions55 +
                                 c * final_predictions66 +
                                 d * final_predictions77 +
                                 e * final_predictions82
                         ) / (a + b + c + d + e)

In [37]:
# SciPy optimize provides functions for minimizing (or maximizing) objective functions, possibly subject to constraints.
from scipy.optimize import minimize


def min_func(K):
    ypredtrain = K[0] * ypredtrainA + K[1] * ypredtrainB + K[2] * ypredtrainC + K[3] * final_train_predictions
    return rmse(train.Pawpularity, ypredtrain)


res = minimize(min_func, [1 / 4] * 4, method='Nelder-Mead', tol=1e-6)
K = res.x
res

 final_simplex: (array([[0.18395402, 0.08305837, 0.19805342, 0.55311892],
       [0.18395461, 0.08305828, 0.198053  , 0.55311888],
       [0.18395426, 0.08305813, 0.19805353, 0.55311878],
       [0.1839544 , 0.08305926, 0.19805293, 0.55311815],
       [0.18395469, 0.08305802, 0.19805364, 0.55311845]]), array([16.87297215, 16.87297215, 16.87297215, 16.87297215, 16.87297215]))
           fun: 16.872972151586403
       message: 'Optimization terminated successfully.'
          nfev: 247
           nit: 140
        status: 0
       success: True
             x: array([0.18395402, 0.08305837, 0.19805342, 0.55311892])

In [42]:
ypredtrain = K[0] * ypredtrainA + K[1] * ypredtrainB + K[2] * ypredtrainC + K[3] * final_train_predictions

test['Pawpularity'] = K[0] * ypredtestA + K[1] * ypredtestB + K[2] * ypredtestC + K[3] * final_test_predictions

print('Ensemble weights:', K)
print('Final RMSE:', rmse(train.Pawpularity, ypredtrain))

Ensemble weights: [0.18395402 0.08305837 0.19805342 0.55311892]
Final RMSE: 16.872972151586403


In [39]:
# Not used (Competition only)
#test.head(8)

In [40]:
# Not used (Competition only)
#test[['Id','Pawpularity']].to_csv('submission.csv', index=False)